# MLOps Zoomcamp 2023 - Session #1 - Homework

Author: José Victor

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

* ( ) 16
* ( ) 17
* ( ) 18
* (X) 19

In [2]:
df = pd.read_parquet(path="data/yellow_tripdata_2022-01.parquet")
len(df.columns)

19

## Q2 Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* ( ) 41.45
* (X) 46.45
* ( ) 51.45
* ( ) 56.45

In [3]:
df['duration'] = df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds() / 60

In [4]:
df['duration'].std()

46.44530513776802

## Q3 Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* ( ) 90%
* ( ) 92%
* ( ) 95%
* (X) 98%

In [5]:
total_of_records = len(df['duration'])

In [6]:
total_of_records

2463931

In [7]:
df_filtered = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [8]:
len(df_filtered)

2421440

In [9]:
(len(df_filtered)/total_of_records)*100

98.27547930522405

## Q4 One-hot encoding

Let's apply one-hot encoding to the pickup and dropff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* ( ) 2
* ( ) 155
* ( ) 345
* (X) 515
* ( ) 715

In [10]:
df_filtered[['PULocationID', 'DOLocationID']] = df_filtered[['PULocationID', 'DOLocationID']].astype(str)

In [11]:
categorical_dict = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(categorical_dict)

In [13]:
X_train.shape

(2421440, 515)

## Q5 Training a model

Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* (X) 6.99
* ( ) 11.99
* ( ) 16.99
* ( ) 21.99

In [14]:
y_train = df_filtered['duration'].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
np.sqrt(mean_squared_error(y_true=y_train, y_pred=lr.predict(X_train)))

6.9861907482152406

## Q6 Evaluating the model

Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

* (X) 7.79
* ( ) 12.79
* ( ) 17.79
* ( ) 22.79

In [17]:
df_val = pd.read_parquet(path="data/yellow_tripdata_2022-02.parquet")

In [18]:
df_val['duration'] = df_val['tpep_dropoff_datetime']-df_val['tpep_pickup_datetime']
df_val['duration'] = df_val['duration'].dt.total_seconds() / 60

In [19]:
df_val_filtered = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)].copy()

In [20]:
df_val_filtered[['PULocationID', 'DOLocationID']] = df_val_filtered[['PULocationID', 'DOLocationID']].astype(str)

In [21]:
categorical_dict = df_val_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [22]:
X_val = dv.transform(categorical_dict)

In [24]:
y_val = df_val_filtered['duration'].values

In [25]:
np.sqrt(mean_squared_error(y_true=y_val, y_pred=lr.predict(X_val)))

7.786406459362245